In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm

In [2]:
DIR = "img"
if not os.path.exists(DIR):
    os.makedirs(DIR)

DIR = "csv"
if not os.path.exists(DIR):
    os.makedirs(DIR)

In [3]:
n = 100
lx = 1
ly = 1
v0 = 0.1
r = 0.05
dt = 0.1
t = 0
imax = 100
eta = np.pi*0.1

vars = pd.DataFrame()
vars["x"] = np.random.random(n) * lx
vars["y"] = np.random.random(n) * ly
vars["theta"] = np.random.random(n) * 2 * np.pi

In [4]:
def plot_points(vars, t, savei=-1):
  plt.rcParams["figure.figsize"] = (6,5)
  plt.rcParams["font.size"] = 12
  plt.rcParams["figure.dpi"] = 120
  plt.xlim(0,lx)
  plt.ylim(0,ly)
  plt.quiver(vars["x"],vars["y"],v0*np.cos(vars["theta"]),v0*np.sin(vars["theta"]),alpha=0.6)
  plt.scatter(vars["x"],vars["y"],alpha=0.4)
  plt.grid(True)
  plt.title(f"time : {t:.3f}")
  if savei >= 0:
    plt.savefig(f'./img/{savei}.png')
  plt.close()

In [5]:
def output_points(vars, t, savei=-1):
  vars.to_csv(f"./csv/{savei}_{t:.3f}.csv",index=False)

In [6]:
def expand_boundary(vars):
  varsex = vars.copy()
  ## upper left
  tmp = vars.copy()
  tmp["x"] -= lx
  tmp["y"] += ly
  tmp = tmp[(tmp["x"] > -r)&(tmp["y"] < ly+r)]
  varsex = pd.concat([varsex,tmp])
  ## upper middle
  tmp = vars.copy()
  tmp["x"] += 0
  tmp["y"] += ly
  tmp = tmp[tmp["y"] < ly+r]
  varsex = pd.concat([varsex,tmp])
  ## upper right
  tmp = vars.copy()
  tmp["x"] += lx
  tmp["y"] += ly
  tmp = tmp[(tmp["x"] < lx+r)&(tmp["y"] < ly+r)]
  varsex = pd.concat([varsex,tmp])
  ## middle left
  tmp = vars.copy()
  tmp["x"] -= lx
  tmp["y"] += 0
  tmp = tmp[tmp["x"] > -r]
  varsex = pd.concat([varsex,tmp])
  ## middle right
  tmp = vars.copy()
  tmp["x"] += lx
  tmp["y"] += 0
  tmp = tmp[tmp["x"] < lx+r]
  varsex = pd.concat([varsex,tmp])
  ## lower left
  tmp = vars.copy()
  tmp["x"] -= lx
  tmp["y"] -= ly
  tmp = tmp[(tmp["x"] > -r)&(tmp["y"] > -r)]
  varsex = pd.concat([varsex,tmp])
  ## lower middle
  tmp = vars.copy()
  tmp["x"] += 0
  tmp["y"] -= ly
  tmp = tmp[tmp["y"] > -r]
  varsex = pd.concat([varsex,tmp])
  ## lower right
  tmp = vars.copy()
  tmp["x"] += lx
  tmp["y"] -= ly
  tmp = tmp[(tmp["x"] < lx+r)&(tmp["y"] > -r)]
  varsex = pd.concat([varsex,tmp])
  return varsex.reset_index(drop=True)

In [7]:
def trim_boundary(vars):
  vars = vars[(vars["x"]>=0)&(vars["x"]<=lx)]
  vars = vars[(vars["y"]>=0)&(vars["y"]<=ly)]
  return vars.reset_index(drop=True)

In [8]:
def update_vars(vars, t):
  vars = expand_boundary(vars)
  matrix = vars["x"].apply(lambda x : (vars["x"]-x)**2) + vars["y"].apply(lambda y : (vars["y"]-y)**2)
  matrix = matrix < r**2
  matrix = matrix.apply(lambda x : vars[x]["theta"])
  vars["theta"] = np.angle(np.exp(matrix*1j).mean())
  vars["theta"] += (np.random.random(len(vars))-0.5) * eta
  vars["x"] += np.cos(vars["theta"]) * v0 * dt
  vars["y"] += np.sin(vars["theta"]) * v0 * dt
  vars = trim_boundary(vars)
  t += dt
  return vars, t

In [9]:
plot_points(vars, t, 0)
output_points(vars, t, 0)
for i in tqdm(range(imax)):
  i += 1
  vars, t = update_vars(vars, t)
  plot_points(vars, t, savei=i)
  output_points(vars, t, savei=i)

100%|██████████| 100/100 [00:15<00:00,  6.40it/s]


In [10]:
import cv2
name                  = "./img/movie.mp4"
filename              = "./img/{}.png"
height, width, layers = cv2.imread(filename.format(imax)).shape
size                  = (width, height)
fourcc                = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
fps = 24
out = cv2.VideoWriter(name, fourcc, fps, size)

for i in range(imax+1):
    img = cv2.imread(filename.format(i))
    out.write(img)
out.release()